# Chapter 4 - 텍스트와 바이트

문자열은 문자의 열이다. 그럼 '문자'는?<br>
현재 문자를 가장 잘 정의한 것은 유니코드 문자이다. 유니코드 표준은 **문자의 단위 원소**와 **특정 바이트 표현**을 명확히 구분한다.<br>

#### **문자의 단위 원소(코드 포인트)**
10진수 0부터 1114111까지의 숫자이며 유니코드 표준에서는 'U+' 접두사를 붙여 4자리에서 6자리 사이의 16진수로 표현한다.

#### **특정 바이트 표현**
문자를 표현하는 실제 바이트는 사용하는 **인코딩**(코드 포인트를 바이트 시퀀스로 변환하는 알고리즘)에 따라 달라진다.

코드 포인트 -> 바이트 (인코딩)<br>
바이트 -> 코드 포인트 (디코딩)<br>

## 바이트에 대한 기본 지식

In [1]:
s1 = 'abcd'
print(type(s1[0]), s1[0]) # 0번 인덱스 원소에 접근해도 자료형은 str
print(type(s1[:1]), s1[:1]) # 슬라이싱해도 자료형은 str
print()

s2 = bytes('café', encoding='utf_8')
print(s2) # 바이트 자료형
print(type(s2[0]), s2[0]) # 0번 인덱스 원소에 접근하면 자료형은 int
print(type(s2[:1]), s2[:1]) # 슬라이싱하면 자료형은 bytes
print()

s3 = bytearray(s2)
print(s3) # 바이트 어레이
print(type(s3[0]), s3[0]) # 위와 동일
print(type(s3[:1]), s3[:1]) # 위와 동일

<class 'str'> a
<class 'str'> a

b'caf\xc3\xa9'
<class 'int'> 99
<class 'bytes'> b'c'

bytearray(b'caf\xc3\xa9')
<class 'int'> 99
<class 'bytearray'> bytearray(b'c')


In [2]:
for i in s2:
    print(i, end=' ') # 이진 시퀀스는 실제로 정수형의 시퀀스
    
print()
print(s2) # 리터럴 표기법에 의하면 화면에 출력가능한 아스키 문자는 아스키 문자 그대로,
          # 탭, 개행문자, 캐리지 리턴, 백슬래시는 이스케이프 시퀀스로,
          # 그 외의 값은 16진수 이스케이프 시퀀스로 출력한다

99 97 102 195 169 
b'caf\xc3\xa9'


## 인코딩/디코팅 문제 이해하기

- UnicodeEncodeError 처리하기

In [3]:
city = 'São Paulo'
print(city.encode('utf_8'))
print(city.encode('utf_16'))
print(city.encode('iso8859_1'))
print(city.encode('cp437'))

b'S\xc3\xa3o Paulo'
b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'
b'S\xe3o Paulo'


UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>

In [4]:
print(city.encode('cp437', errors='ignore'))
print(city.encode('cp437', errors='replace'))
print(city.encode('cp437', errors='xmlcharrefreplace'))

b'So Paulo'
b'S?o Paulo'
b'S&#227;o Paulo'


- UnicodeDecodeError 처리하기

In [5]:
octets = b'Montr\xe9al'
print(octets.decode('cp1252'))
print(octets.decode('iso8859_7'))
print(octets.decode('koi8_r'))
print(octets.decode('utf8'))

Montréal
Montrιal
MontrИal


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [6]:
print(octets.decode('utf8', errors='replace'))

Montr�al


- BOM: 유용한 깨진 문자

BOM은 바이트 순서 표시(Byte Order Mark)로, 리틀엔디언/빅엔디언으로 발생하는 혼란을 방지하기 위해 UTF-16 인코딩은 ZWNBSP(Zero Width No-Break SPace, U+FEFF)를 인코딩된 텍스트 앞에 붙이는데, 이 문자는 화면에 출력되지 않는다.<br>
리틀엔디언 컴퓨터에서는 b'\\xff\\xfe'로 인코딩되는데, UTF-16에 U+FFFE에 해당하는 문자는 없으므로 리틀엔디언으로 인코딩 되었음을 알 수 있다.<br>

## 기본 인코딩에 의존하지 말라

유니코드 샌드위치 모델을 따를 것. 그리고 프로그램 안에서 늘 인코딩을 명시할 것.

## 제대로 비교하기 위해 유니코드 정규화하기

코드 포인트 U+0301은 COMBINING ACUTE ACCENT이며 'e' 다음에 U+0301이 오면 'é'를 만든다.<br>
유니코드 표준에서는 'é'와 'e\u0301' 두 개의 시퀀스를 '규범적으로 동일하다'고 하며, 애플리케이션은 이 두 시퀀스를 동일하게 처리해야 한다.<br>
그렇지만 파이썬은 서로 다른 두 개의 시퀀스로 보고, 이 둘을 서로 동일하지 않다고 판단한다.

In [20]:
from unicodedata import normalize

s1 = 'café'
s2 = 'cafe\u0301'

print(s1, s2)
print(normalize('NFC', s1), normalize('NFC', s2))
print(normalize('NFD', s1), normalize('NFD', s2))
print()

print(len(s1), len(s2))
print(len(normalize('NFC', s1)), len(normalize('NFC', s2)))
print(len(normalize('NFD', s1)), len(normalize('NFD', s2)))
print()

print(s1 == s2)
print(normalize('NFC', s1) == normalize('NFC', s2))
print(normalize('NFD', s1) == normalize('NFD', s2))

café café
café café
café café

4 5
4 4
5 5

False
True
True


## 유니코드 텍스트 정렬하기

문자열의 경우 각 단어의 코드 포인터를 비교하는데, 불행히도 이런 방식은 비아스키 문자를 사용하는 경우 부적절한 결과가 발생할 수 있다.

In [1]:
fruits = ['açaí', 'acerola', 'atemoia', 'cajá', 'caju']
print(sorted(fruits))

['acerola', 'atemoia', 'açaí', 'caju', 'cajá']


포르투갈어 등 라틴 알파벳을 사용하는 언어에서는 정렬할 때 악센트와 갈고리형 기호가 거의 영향을 미치지 않는다.<br>
따라서 'cajá'는 'caja'로 처리하여 'caju'보다 먼저 나와야 한다.<br>
이는 PyUCA를 사용하면 된다.

In [2]:
import pyuca # 모듈이 없으면 설치해야 한다
coll = pyuca.Collator()

sorted_fruits = sorted(fruits, key=coll.sort_key)
print(sorted_fruits)

ModuleNotFoundError: No module named 'pyuca'